[Example Notebook](https://colab.research.google.com/drive/1bPhY1Awz0NIuzGy9RX72eZRB10FwZY0j?usp=sharing#scrollTo=6o7h7-SdMKHK)

In [1]:
import os
import requests
import json
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages 
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from IPython.display import Markdown
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

## Normal Execution

In [3]:
prompt = """GENERAL INSTRUCTIONS
You are a domain expert. Your task is to break down a complex question into simpler sub-parts so that it will be 
easier to answer a complex question. Think of it as step by step approach to reach to a solution.

USER QUESTION
{user_question}

ANSWER FORMAT
Provide sub questions in dictionary format with key as "sub-questions" and sub questions in list as value of the dictionary
"""

In [4]:
prompt = """
You are a helpful assistant which answers user's questions. For a complex question which you think
you need some help, then you can request some help from available help or use tool from available tool.

USER QUESTION
{user_question}

AVAIALABLE TOOL
1) Search Tool
2) Math Tool

AVAILABLE HELP
Decomposition: Breaks complex question into simple sub questions to reach to the final answer

Provide final answer in a dictionary format with keys as 'Tool_Request' and 'Helper_Request'

Update the value of the key if you used the help or the tool. If you doesn't use then update it with "Not Required".

NO PREAMBLE NEEDED.
"""

In [5]:
prompt_template = ChatPromptTemplate.from_template(prompt)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.environ["OPENAI_API_KEY"])

chain = prompt_template | llm | StrOutputParser()

In [6]:
json.loads(chain.invoke("how much did the revenue increaases for Apple between Q1 of 2024 and Q2 of 2024?"))

{'Tool_Request': 'Search Tool', 'Helper_Request': 'Not Required'}

## Started with Tools

In [7]:
# first tool -  Calculator

@tool
def calculate(operation):
    """Useful to perform any mathematical calculations,
    like sum, minus, multiplication, division, etc.
    The input to this tool should be a mathematical
    expression, a couple examples are `200*7` or `5000/2*10`
    """
    try:
        return eval(operation)
    except SyntaxError:
        return "Error: Invalid syntax in mathematical expression"
    
# Second tool - Search tool

@tool
def search_internet(query):
    """Useful to search the internet
        about a a given topic and return relevant results"""
    try:
        top_result_to_return = 4
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        # check if there is an organic key
        if 'organic' not in response.json():
            return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
        else:
            results = response.json()['organic']
            string = []
            for result in results[:top_result_to_return]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}", f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}", "\n-----------------"
                    ]))
                except KeyError:
                    next

            return '\n'.join(string)
    except Exception:
        print("Exception occurred in search tool")


In [8]:
available_tools = [
    calculate,
    search_internet
]

functions = [convert_to_openai_function(f) for f in available_tools]

In [9]:
functions

[{'name': 'calculate',
  'description': 'Useful to perform any mathematical calculations,\n    like sum, minus, multiplication, division, etc.\n    The input to this tool should be a mathematical\n    expression, a couple examples are `200*7` or `5000/2*10`',
  'parameters': {'properties': {'operation': {}},
   'required': ['operation'],
   'type': 'object'}},
 {'name': 'search_internet',
  'description': 'Useful to search the internet\n        about a a given topic and return relevant results',
  'parameters': {'properties': {'query': {}},
   'required': ['query'],
   'type': 'object'}}]

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.environ["OPENAI_API_KEY"])

llm_with_tools = llm.bind(functions=functions)

In [11]:
prompt = ChatPromptTemplate.from_messages(
[
    ("system","you are a helpful AI assistant which helps users to resolve their queries"),
    ("user", "{question}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [12]:
chain = prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [13]:
agent = (
{
    "question": lambda x: x["question"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser()
)


In [14]:
agent_executor = AgentExecutor(agent=agent, tools=available_tools, verbose=True)

In [15]:
agent_executor.invoke({"question": "Can you provide me the difference between the revenue of Google between Q1 2024 and Q2 2024?"})



> Entering new AgentExecutor chain...

Invoking: `search_internet` with `{'query': 'Google revenue Q1 2024 Q2 2024'}`


Title: [PDF] Alphabet Announces Second Quarter 2024 Results
Link: https://abc.xyz/assets/19/e4/3dc1d4d6439c81206370167db1bd/2024q2-alphabet-earnings-release.pdf
Snippet: Ruth Porat, President and Chief Investment Officer; CFO said: “We delivered revenues of $85 billion, up 14% year- on-year driven by Search as ...

-----------------
Title: [PDF] GOOG Exhibit 99.1 Q1 2024 - Alphabet Inc.
Link: https://abc.xyz/assets/91/b3/3f9213d14ce3ae27e1038e01a0e0/2024q1-alphabet-earnings-release-pdf.pdf
Snippet: Google Cloud generates revenues primarily from consumption-based fees and subscriptions received for Google Cloud Platform services, Google ...

-----------------
Title: Alphabet Announces Second Quarter 2024 Results - SEC.gov
Link: https://www.sec.gov/Archives/edgar/data/1652044/000165204424000076/googexhibit991q22024.htm
Snippet: We delivered revenues of $85 billion, up

{'question': 'Can you provide me the difference between the revenue of Google between Q1 2024 and Q2 2024?',
 'output': 'In Q1 2024, Google (Alphabet Inc.) reported revenues of **$80.5 billion**. In Q2 2024, the revenue was **$85 billion**. \n\nTo find the difference in revenue between Q2 2024 and Q1 2024:\n\n\\[\n\\text{Difference} = \\text{Revenue in Q2 2024} - \\text{Revenue in Q1 2024} = 85 \\text{ billion} - 80.5 \\text{ billion} = 4.5 \\text{ billion}\n\\]\n\nThus, the difference in revenue between Q2 2024 and Q1 2024 is **$4.5 billion**.'}

In [16]:
agent_executor.invoke(
    {
        "question": """Hello there"""
    }
)



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'question': 'Hello there', 'output': 'Hello! How can I assist you today?'}

In [17]:
class CustomAgent:

    def __init__(self) -> None:
        self.OPENAI_LLM =  ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.environ["OPENAI_API_KEY"])
        self.GROQ_LLM = ChatGroq(model="llama-3.1-70b-versatile", api_key=os.environ["GROQ_API_KEY"], temperature=0)

    @staticmethod
    @tool
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Error: Invalid syntax in mathematical expression"
        

    @staticmethod
    @tool
    def search_internet(query):
        """Useful to search the internet
            about a a given topic and return relevant results"""
        try:
            top_result_to_return = 4
            url = "https://google.serper.dev/search"
            payload = json.dumps({"q": query})
            headers = {
                'X-API-KEY': os.environ['SERPER_API_KEY'],
                'content-type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            # check if there is an organic key
            if 'organic' not in response.json():
                return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
            else:
                results = response.json()['organic']
                string = []
                for result in results[:top_result_to_return]:
                    try:
                        string.append('\n'.join([
                            f"Title: {result['title']}", f"Link: {result['link']}",
                            f"Snippet: {result['snippet']}", "\n-----------------"
                        ]))
                    except KeyError:
                        next

                return '\n'.join(string)
        except Exception:
            print("Exception occurred in search tool")

    
    def get_agent_output(self, question: str):

        available_tools = [
            calculate,
            search_internet
        ]

        functions = [convert_to_openai_function(f) for f in available_tools]

        llm = self.OPENAI_LLM

        llm_with_tools = llm.bind(functions=functions)

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system","you are a helpful AI assistant which helps users to resolve their queries"),
                ("user", "{question}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ])
        
        agent = (
        {
            "question": lambda x: x["question"],
            "agent_scratchpad": lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | OpenAIFunctionsAgentOutputParser())


        agent_executor = AgentExecutor(agent=agent, tools=available_tools)

        return agent_executor.invoke({"question": question})


In [18]:
custom_agent = CustomAgent()

In [19]:
user_question = "Can you provide me the difference between the revenue of Google between Q1 2024 and Q2 2024?"

In [20]:
output = custom_agent.get_agent_output(
    question=user_question
)

In [21]:
Markdown(output["output"])

The revenue for Google (Alphabet Inc.) in Q1 2024 was $80.5 billion, while in Q2 2024, it was $84.7 billion. 

To find the difference in revenue between Q1 2024 and Q2 2024:

\[
\text{Difference} = \text{Q2 Revenue} - \text{Q1 Revenue} = 84.7 \, \text{billion} - 80.5 \, \text{billion} = 4.2 \, \text{billion}
\]

Thus, the difference in revenue between Q1 2024 and Q2 2024 is $4.2 billion.

In [22]:
user_question = """
Can you comment on the on going war between Iran and Israel?\
Why these 2 nations are fighting each other? Will this lead to World War 3?\
What's your opninion here?
"""

output = custom_agent.get_agent_output(
    question=user_question
)

In [23]:
Markdown(output["output"])

The ongoing conflict between Iran and Israel is deeply rooted in historical, political, and ideological differences. Here are some key points regarding the situation:

1. **Historical Context**: The animosity between Iran and Israel has been escalating since the 1979 Iranian Revolution, which led to the establishment of the Islamic Republic of Iran. Iran's leadership views Israel as a primary adversary, often referring to it as an "illegitimate" state.

2. **Current Escalation**: Recent reports indicate that tensions have significantly increased, with Iran launching missile attacks against Israel. This is part of a broader pattern of hostilities that includes proxy conflicts involving groups like Hezbollah in Lebanon and Hamas in Gaza, which are supported by Iran.

3. **Regional Dynamics**: The conflict is not just bilateral; it involves various regional players and has implications for global security. The involvement of other nations, including the United States and various Arab states, complicates the situation further.

4. **Potential for Wider Conflict**: While the situation is tense, predicting a full-scale World War III is speculative. However, the risk of escalation into a broader conflict exists, especially if other nations become involved or if there are significant miscalculations on either side.

5. **Conclusion**: The conflict between Iran and Israel is complex and multifaceted, driven by a mix of ideological, political, and military factors. While the potential for escalation is real, the international community is likely to seek diplomatic solutions to prevent a wider war.

In summary, the situation remains fluid, and while it poses significant risks, the outcome will depend on various factors, including diplomatic efforts and the actions of other regional and global powers.

In [24]:
user_question = """
what are the ongoing issues going for canada?
"""

output = custom_agent.get_agent_output(
    question=user_question
)

In [25]:
Markdown(output["output"])

As of 2023, Canada is facing several ongoing issues, including:

1. **Income Inequality**: There has been a reported increase in income inequality, which has been a growing concern as the country emerges from the COVID-19 pandemic.

2. **Housing Crisis**: Issues related to low income, overcrowded or inadequate housing, and lack of access to affordable housing are significant challenges.

3. **Education Disparities**: A lack of high school diplomas among certain populations is contributing to ongoing social issues.

4. **Racism and Discrimination**: The Canadian government is actively working on an Anti-Racism Strategy to combat increasing levels of racism and discrimination.

5. **Economic Challenges**: There are concerns about skyrocketing national debt and falling living standards, which are affecting the overall economic stability of the country.

6. **Sustainable Development Goals**: Canada is also focused on addressing various social, economic, and environmental challenges as part of its commitment to the Sustainable Development Goals.

These issues reflect a complex landscape that the country is navigating in 2023. For more detailed information, you can refer to the sources linked in the search results.

## Using CrewAI

### Simple Agent

In [27]:
from crewai import Agent, Crew, Process, Task, LLM

In [29]:
# using Llama 3.3 from Groq
llama_llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.7
)

In [48]:
simple_agent = Agent(
    role="you are a helpful AI agent responds to the user's queries",
    goal="Your goal is to understand the user query completely and then respond with a answer which the user is looking for",
    backstory="""
    You are a helpful AI Agent which helps to resolve the user's query. The user might face some issue
    and you need to provide resolution best to your knowledge. The answer should be accurate and doesn't contain any error
    """,
    llm=llama_llm  # Assign the LLM to the agent
)

In [49]:
# Define a task for the agent
task = Task(
    description="Respond to the user's query: {input}",
    agent=simple_agent,
    expected_output="An accurate answer to the user's query"
)


In [50]:
crew = Crew(
    agents=[simple_agent],
    tasks=[task]
)

In [52]:
result = crew.kickoff({"input":"what is the 2+2?"})

In [53]:
print(result)

The answer to the equation 2+2 is 4.
